In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader,random_split,Dataset

In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [3]:
import os
c= 0;
texts = []
labels = []
for i in range(1,9):
    dirname = f'/content/drive/MyDrive/Golestan/gbab{i}'
    for j in range(1,len(os.listdir(dirname))+1):
        filename = dirname + '/'+f'hekayat{j}.txt'
        with open(filename,encoding='utf-8') as f :
            texts.append(f.readlines()[0]);
            labels.append(i-1)


In [4]:
import re
texts = [re.sub(r'[\u064B-\u0652]','',t) for t in texts];
texts[0]

'پادشاهی را شنیدم به کشتن اسیری اشارت کرد بی چاره در آن حالت نومیدی ملک را دشنام دادن گرفت و سقط گفتن که گفته اند هر که دست از جان بشوید هر چه در دل دارد بگوید وقت ضرورت چو نماند گریز دست بگیرد سر شمشیر تیز إذا یئس الإنسان طال لسانه کسنور مغلوب یصول علی الکلب ملک پرسید چه می گوید یکی از وزرای نیک محضر گفت ای خداوند همی گوید و الکاظمین الغیظ و العافین عن الناس ملک را رحمت آمد و از سر خون او درگذشت وزیر دیگر که ضد او بود گفت ابنای جنس ما را نشاید در حضرت پادشاهان جز به راستی سخن گفتن این ملک را دشنام داد و ناسزا گفت ملک روی از این سخن در هم آورد و گفت آن دروغ وی پسندیده تر آمد مرا زین راست که تو گفتی که روی آن در مصلحتی بود و بنای این بر خبثی و خردمندان گفته اند دروغی مصلحت آمیز به که راستی فتنه انگیز هر که شاه آن کند که او گوید حیف باشد که جز نکو گوید بر طاق ایوان فریدون نبشته بود جهان ای برادر نماند به کس دل اندر جهان آفرین بند و بس مکن تکیه بر ملک دنیا و پشت که بسیار کس چون تو پرورد و کشت چو آهنگ رفتن کند جان پاک چه بر تخت مردن چه بر روی خاک '

In [5]:
with open('/content/drive/MyDrive/Golestan/stopwords.txt',encoding='utf-8') as f :
    stopwords = set(f.read().split('\n'))

In [6]:
model_name = 'HooshvareLab/bert-fa-base-uncased';
tokenizer = AutoTokenizer.from_pretrained(model_name) ;

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
class Text(Dataset):
  def __init__(self, texts,labels,tokenizer):
    super().__init__();
    self.texts = texts;
    self.tokenizer = tokenizer ;
    self.labels = labels;
  def __len__(self) : return len(self.texts);
  def __getitem__(self, index):
    text = self.texts[index];
    label = self.labels[index];
    tokens = self.tokenizer(text,truncation=True,max_length = 512,padding='max_length',
                            return_tensors='pt');
    item = {key:val.squeeze(0) for key,val in tokens.items()};
    item['labels'] = torch.tensor(label,dtype=torch.int64);
    return item

In [8]:
dataset = Text(texts,labels,tokenizer);
dataset[0]

{'input_ids': tensor([    2,  6131,  2803, 18426,  2789,  9745, 45564, 54965,  2830,  2883,
         15251,  2786,  2808,  4157, 48198,  6722,  2803, 38060,  4186,  3153,
          1379, 14067,  8540,  2800,  3538,  3145,  2937,  2800,  2910,  2791,
          3607, 19718,  2937,  3051,  2786,  3217,  2924,  9531,  4244,  7109,
         16176, 11659, 12120,  2910,  4962,  2892, 10626, 10426, 36766, 89480,
          2017, 78595,  8342, 16255,  2008, 73739,  2795, 12670,  1442,  3516,
          3366,  9354,  2016,  6722, 10840,  3051,  2793, 18889,  3036,  2791,
         10224,  7513, 15513,  3017,  2938,  6512, 27131, 18889,  1379, 70372,
          2788, 78592,  2062,  1379,  5679, 45570,  5701, 25289,  6722,  2803,
         13823,  3771,  1379,  2791,  2892,  4266,  2866,  5661,  3646,  2972,
          2800,  4698,  2866,  2834,  3017, 98935,  6586,  2964,  2803, 41265,
          2805,  2786,  5883, 10487,  3430,  2789, 12320,  4402,  8540,  2802,
          6722,  2803, 38060,  2974,  1

In [9]:
torch.cuda.empty_cache()

In [10]:
train_dataset,val_dataset = random_split(dataset,[250,40]);
train_loader = DataLoader(train_dataset,batch_size=8,shuffle=True);
val_loader = DataLoader(val_dataset,batch_size=8,shuffle=False);

In [11]:
device = 'cuda'

In [12]:
model1 = AutoModelForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased',
                                                            num_labels=8)
model1.to(device);

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = torch.optim.Adam(lr=2e-5,params= model1.parameters());
loss_function = nn.CrossEntropyLoss()

In [14]:
torch.cuda.empty_cache()

In [16]:
for i in range(10):
  model1.train();
  for x in train_loader :
    optimizer.zero_grad();
    attention_mask = x['attention_mask'].to(device);
    input_ids = x['input_ids'].to(device);
    labels = x['labels'].to(device);
    out = model1(attention_mask = attention_mask,labels=labels,input_ids=input_ids);
    logits = out.logits;
    loss = loss_function(logits,labels);
    loss.backward();
    optimizer.step();
  print(f"Epoch {i} ---> Loss : {round(loss.item(),4)}")
# Notice that I ran this cell twice.

Epoch 0 ---> Loss : 0.0373
Epoch 1 ---> Loss : 0.0237
Epoch 2 ---> Loss : 0.0292
Epoch 3 ---> Loss : 0.014
Epoch 4 ---> Loss : 0.0063
Epoch 5 ---> Loss : 0.0194
Epoch 6 ---> Loss : 0.012
Epoch 7 ---> Loss : 0.0081
Epoch 8 ---> Loss : 0.0029
Epoch 9 ---> Loss : 0.0049


In [18]:
from sklearn.metrics import accuracy_score,f1_score
model1.eval();
predicts = []
y_test = []
with torch.no_grad():
  for x in val_loader :
    attention_mask = x['attention_mask'].to(device);
    input_ids = x['input_ids'].to(device);
    labels = x['labels'].to(device);
    out = model1(attention_mask = attention_mask,labels=labels,input_ids=input_ids);
    logits = out.logits;
    predict = logits.argmax(1);
    predicts.extend(predict.tolist());
    y_test.extend(labels.tolist());
print("Accuracy :",accuracy_score(y_test,predicts));
print("F1-score(macro) :",f1_score(y_test,predicts,average='macro'))

Accuracy : 0.5
F1-score(macro) : 0.3135989010989011
